In [ ]:
#kniznice
install.packages("tidyverse")
library(tidyverse) 
library(reshape2)

In [ ]:
#Pocty prenesenych dat:

#set the working directory from which the files will be read from
setwd("data/poctyPrenesenychDat/")

#create a list of the files from your target directory
file_list <- list.files(pattern="*.csv") #musel som pridat nuly pred cisla v nazvoch lebo inak bralo 1,10,11,...

#initiate a blank data frame, each iteration of the loop will append the data from the given file to this variable
dataset_poctyPrenesenychDat <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=",", encoding="UTF-8") 
  dataset_poctyPrenesenychDat <- rbind(dataset_poctyPrenesenychDat, temp_data) #for each iteration, bind the new data to the building dataset
}

#Data su od 24.5.2018 do 29.2.2020. Prvy problem je, ze v podstate nemame udaje pre obdobie pocas korony,
#druhy problem je, ze v niektore dni je viac merani. Treba to spojit v ramci jednotlivych dni. Navrhujem
#predkoronove udaje neskor vygenerovat

#write.csv2(dataset, "../../dataset.csv")

setwd("../..")

#column names na lowercase
names(dataset_poctyPrenesenychDat) <- tolower(names(dataset_poctyPrenesenychDat))
#show(dataset_poctyPrenesenychDat)


In [ ]:
#Udaje o pouzivateloch + dlzka pripojenia:

setwd("data/udajeDlzkaPripojenia/")

file_list <- list.files(pattern="*.csv") 

dataset_udajeDlzkaPripojenia <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=";", encoding="UTF-8") 
  dataset_udajeDlzkaPripojenia <- rbind(dataset_udajeDlzkaPripojenia, temp_data) 
}
#Data su od 1.6.2018 do 25.2.2021. Problemy som nezaznamenal ziadne. Toto bude fajn zaklad pre nas dataset.

#write.csv2(dataset, "../../dataset.csv")

setwd("../..")

#column names na lowercase
names(dataset_udajeDlzkaPripojenia) <- tolower(names(dataset_udajeDlzkaPripojenia))
#show(dataset_udajeDlzkaPripojenia)



In [ ]:
#Udaje o zariadeni - Jazyk, OS, Typ zariadenia, Typ prehliadaca:

setwd("data/osJazykTypPrehliadac/")

file_list <- list.files(pattern="*.csv") 

dataset_osJazykTypPrehliadac <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=";", encoding="UTF-8") 
  dataset_osJazykTypPrehliadac <- rbind(dataset_osJazykTypPrehliadac, temp_data) 
}

#column names na uppercase
names(dataset_osJazykTypPrehliadac) <- tolower(names(dataset_osJazykTypPrehliadac))


#Data su od 24.5.2018 do 25.2.2021. Velmi pekne data, ktore urcite chceme pouzit. Problem je ich forma.
#Je tam variabilny pocet zaznamov pre kazdy den, podla mnozstva identifikovanych jazykov atd v dany den.
#Treba to previest na stlpce a tiez asi zvazit ktore hodnoty nam stoji za to pouzit v datasete a ktore nie. 

#write.csv2(dataset, "../../dataset.csv")
#show(dataset_osJazykTypPrehliadac)
setwd("../..")

In [ ]:
#Pocty podla oblasti:

setwd("data/poctyPodlaOblasti/")

file_list <- list.files(pattern="*.csv")

dataset_poctyPodlaOblasti <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=";", encoding="UTF-8", stringsAsFactors=FALSE) 
  dataset_poctyPodlaOblasti <- rbind(dataset_poctyPodlaOblasti, temp_data) 
}

#Vo vysledom dataframe su najprv udaje po celych mesiacoch a lokality su menej podelene. 
#Nasledne od 1.10.2019 su data po dnoch a "ostatne" je rozdelene na konkretne lokality. Navrhujem teda zatial
#napojit len od 1.10.2019 a pred tymto datumom to neskor podelime na dni tak aby bolo rozdelenie priblizne rovnake 

#column names na uppercase
names(dataset_poctyPodlaOblasti) <- tolower(names(dataset_poctyPodlaOblasti))


#write.csv2(dataset, "../../dataset.csv")
#show(dataset_poctyPodlaOblasti)
setwd("../..")

In [ ]:
#spoji stplpce "type" a "name" => type_name
#type: browser, name: Firefox, value: 5 => type_name: browser_Firefox, value:5
type_and_name <- unite(dataset_osJazykTypPrehliadac, type_name, c(type, name), remove=TRUE)

# prekonvertuje riadok na stlpec, unique ID je date pri dataset_osJazykTypPrehliadac
# type_name: browser_Firefox, value:5 => browser_Firefox: 5 pre dany den
dataset_type_and_name <- dcast(type_and_name, date ~ type_name, value.var = "value")

# prekonvertuje riadok na stlpec, unique ID je date pri poctyPodlaOblasti
dataset_location <- dcast(dataset_poctyPodlaOblasti, date ~ location, value.var = "value")

In [ ]:
#konvertujeme na Date format aby sme mohli nasledne filtrovat
dataset_location$date <- as.Date(dataset_location$date, format="%Y-%m-%d %H:%M:%OS")

#chceme data po 2019-10-01 kedze predtym data boli za cely mesiac 
dataset_filtered_location <- subset(dataset_location, date > "2019-10-01")


In [ ]:
#merge datasets na zaklade date (poctyPrenesenychDat + udajeDlzkaPripojenia)
merged_dataset_1 <- merge(dataset_poctyPrenesenychDat, dataset_udajeDlzkaPripojenia, by="date", all=TRUE)

#merge datasets na zaklade date (poctyPrenesenychDat + udajeDlzkaPripojenia + osJazykTypPrehliadac)
merged_dataset_2 <- merge(merged_dataset_1, dataset_type_and_name, by="date", all=TRUE)

#merge datasets na zaklade date (poctyPrenesenychDat + udajeDlzkaPripojenia + osJazykTypPrehliadac + poctyPodlaOblasti)
final_dataset <- merge(dataset_filtered_location, merged_dataset_2, by="date", all=TRUE)

write.csv2(final_dataset, "../../dataset_final.csv")
head(final_dataset)

